In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

"apt-get" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.
"apt" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.
"cp" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.


In [2]:
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium import common
from tqdm import tqdm
from parsing import get_html

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

C:\Users\aobor\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [4]:
base_url = 'https://market.chelpipe.ru/'
base_html = get_html(base_url)

# browser.implicitly_wait(0.5)
main_list = list()
a = '/production/truby-stalnye/'
page = 1
new_url = (base_url + a).replace('//', '/')
browser.get(new_url)
while True:
    try:
        table = browser.find_element_by_class_name('table__main')
        print(page)
    except:
        break

    table_html = table.get_attribute('outerHTML')
    table_soup = BeautifulSoup(table_html, 'lxml')
    div_row = table_soup.find_all('div', {'class': 'table-row'})
    for dr in div_row:

        page_product = dr.find('a').get('href')

        try:
            browser.get((base_url + page_product).replace('//', '/'))
        except common.exceptions.TimeoutException:
            print('TimeoutException')
            print((base_url + page_product).replace('//', '/'))
            browser.get((base_url + page_product).replace('//', '/'))
        try:
            product_page = browser.find_element_by_class_name('product-card-main__stats').get_attribute('outerHTML')
        except common.exceptions.NoSuchElementException:
            print('NoSuchElementException')
            print(browser.current_url)
            product_page = browser.find_element_by_class_name('product-card-main__stats').get_attribute('outerHTML')
        cart = BeautifulSoup(product_page, 'lxml')
        data = cart.find_all('div', {'class': 'product-card-main__stats-item-value'})

        production_method = data[5].text
        if production_method == 'Горячедеформированные трубы':
          production_method = 'г/д'
        elif production_method == 'Холоднодеформированные трубы':
          production_method = 'х/д'
        elif production_method == 'Электросварные трубы':
          production_method = 'ЭСВ'
        else:
          production_method = None

        article = data[-1].text
        type_of_length = data[-2].text.strip()

        row = dr.find_all('div', {'class': 'table-row__item-value'})

        for_tube = dr.find('div', {'class': 'table-row__item-value-secondary'})
        type_tube = for_tube.find_all('div')[-1].text.strip()
        standard = dr.find('div', {'class': 'table-row__item-value-primary'}).text.split(',')
        steel = row[0].text
        diameter = float(row[1].text.split('/')[0].strip())
        wall = float(row[2].text)
        region = row[3].text
        a_r = row[4].find('span').text.split('/')

        try:
            availability = float(a_r[0].strip())
        except ValueError:
            availability = 0

        try:
            if len(a_r) > 1:
                reserve = float(a_r[-1].strip())
            else:
                reserve = 0

        except ValueError:
            raise ValueError()
        float_prices = []
        if 'от' in row[5].text:
          price_table =  browser.find_element_by_class_name("new-table__inner").get_attribute('outerHTML')
          price_table = BeautifulSoup(price_table, 'lxml')
          price_table = price_table.find_all('div', {'class': 'new-table__row'})[1]
          prices = price_table.find_all('div', {'class': 'new-table__cell--order-1'})
          for pr in prices:
            buff_price = pr.find('div', {'class': 'new-table__cell-value'}).text.replace(u'\xa0', u'').replace('руб', '').strip()
            if buff_price == '—':
              float_prices.append(0)
            else:
              float_prices.append(float(buff_price))
        else:
          try:
              price = float(row[5].text.replace(u'\xa0', u'').replace('i', '').replace('от', '').replace('руб.', '').strip())
          except ValueError:
              price = 0

        for s in standard:
            main_list.append({'type_tube': type_tube, 
                              'standard': s.replace('\n', '').strip(),
                              'steel': steel,
                              'diameter': diameter,
                              'wall': wall,
                              'region': region,
                              'availability': availability,
                              'reserve': reserve,
                              'price_min': float_prices[0] if float_prices else price,
                              'price_max': float_prices[-1] if float_prices else None,
                              'article': article,
                              'type_of_length': type_of_length,
                              'production_method': production_method})

    page += 1
    browser.get(new_url + f'?PAGEN_1={page}')

C:\Users\aobor\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: headless chrome=87.0.4280.88)


In [ ]:
json_list = json.dumps(main_list,  ensure_ascii=False).replace('},', '},\n')
with open('truby-stalnye.json', "w",  encoding='utf-8') as file:
    file.write(json_list)
browser.quit()